In [ ]:
import random
import math

courseData = [{"name": "SE", "type": "Theory", "weeklyLectures": 2}, {"name": "Web", "type": "Theory", "weeklyLectures": 2}, {"name": "Algo", "type": "Theory", "weeklyLectures": 2}, {"name": "LA", "type": "Theory", "weeklyLectures": 2}, {"name": "DLD", "type": "Theory", "weeklyLectures": 2}, {"name": "DLD - Lab", "type": "Lab", "weeklyLectures": 1}, {"name": "OOP", "type": "Theory", "weeklyLectures": 2}, {"name": "OOP - Lab", "type": "Lab", "weeklyLectures": 1}, {"name": "COAL", "type": "Theory", "weeklyLectures": 2}, {"name": "DS", "type": "Theory", "weeklyLectures": 2}, {"name": "DS - Lab", "type": "Lab", "weeklyLectures": 1}, {"name": "Prob", "type": "Theory", "weeklyLectures": 2}, {"name": "Islamiat", "type": "Theory", "weeklyLectures": 2}, {"name": "FOM", "type": "Theory", "weeklyLectures": 2}, {"name": "DB", "type": "Theory", "weeklyLectures": 2}, {"name": "DB - Lab", "type": "Lab", "weeklyLectures": 1}]

sectionCourses = {"A": ["DLD", "OOP", "COAL", "LA"], "B": ["Web", "Algo", "DS", "Islamiat"], "C": ["FOM", "DB", "Prob", "SE"], "D": ["DLD - Lab", "OOP - Lab", "DS - Lab", "DB - Lab"], "E": ["DLD", "OOP", "COAL", "LA"], "F": ["Web", "Algo", "DS", "Islamiat"], "G": ["FOM", "DB", "Prob", "SE"], "H": ["DLD - Lab", "OOP - Lab", "DS - Lab", "DB - Lab"], "I": ["DLD", "OOP", "COAL", "LA"]}

sectionInfo = {"A": {"strength": 42}, "B": {"strength": 48}, "C": {"strength": 71}, "D": {"strength": 72}, "E": {"strength": 52}, "F": {"strength": 43}, "G": {"strength": 88}, "H": {"strength": 91}, "I": {"strength": 58}}

roomData = {"C301": {"capacity": 60, "isFree": [True] * 12}, "C302": {"capacity": 60, "isFree": [True] * 12}, "C303": {"capacity": 60, "isFree": [True] * 12}, "C304": {"capacity": 60, "isFree": [True] * 12}, "C305": {"capacity": 60, "isFree": [True] * 12}, "C306": {"capacity": 60, "isFree": [True] * 12}, "C501": {"capacity": 120, "isFree": [True] * 12}, "C502": {"capacity": 120, "isFree": [True] * 12}, "C503": {"capacity": 120, "isFree": [True] * 12}}

teacherList = ["Mr. Irfanullah", "Mr. Muhammad Ali", "Mr. Shams Farooq", "Ms. Labiba Fahad", "Mr. Majid Hussain", "Mr. Shahnawaz", "Mr. Owais Idrees", "Mr. Almas", "Mr. Aleem"]

num_days = 5
timeslots_per_day = 6

def timeslot_to_time(timeslot):
    start_hour = 8  # Hour part of the start time
    start_minute = 30 # Minute part of the start time
    slot_duration = 80  # 1 hour 20 minutes
    break_duration = 15
    total_minutes = (start_hour * 60) + start_minute + (timeslot - 1) * (slot_duration + break_duration)
    hour = total_minutes // 60
    minute = total_minutes % 60
    return f"{hour:02d}:{minute:02d}"

# Define binary encoding lengths for each field
encoding_lengths = {
    "Course": 5,
    "Type": 1,
    "Section": 3,
    "SectionStrength": 6,
    "Professor": 4,
    "Day": 3,
    "Timeslot": 3,
    "Room": 4,
    "RoomSize": 6,
}

# Checks constraint if classroom is free at that timeslot
# Checks constraint if classroom capacity is greater than number of students
def check_classroom_constraints(timetable):
    violations = 0
    for classInfo in timetable:
        classroom = classInfo["Classroom"]
        course = classInfo["Course"]
        section = classInfo["Section"]
        courseType = classInfo["Type"]
        if not roomData[classroom]["isFree"][classInfo["Timeslot"] - 1]:
            violations += 1
        if roomData[classroom]["capacity"] < sectionInfo[section]["strength"]:
            violations += 1
        if courseType == "Theory" and roomData[classroom]["capacity"] != 60:
            violations += 1
        elif courseType == "Lab" and roomData[classroom]["capacity"] != 120:
            violations += 1
    return violations

# binary encoding
def binary_encode(value, length):
    return bin(value)[2:].zfill(length)


def create_binary_chromosome():
    chromosome = ""
    for field, length in encoding_lengths.items():
        if field in ["Type"]:  # Handle special cases
            if field == "Type" and random.random() < 0.5:
                chromosome += "1"  # Encode Lab as 1
            else:
                chromosome += "0"  # Encode Theory as 0
        elif field in ["Course", "Section", "Professor", "Day", "Timeslot", "Room"]:
            chromosome += binary_encode(random.randint(1, 2**length - 1), length)
        elif field == "SectionStrength":
            chromosome += binary_encode(random.randint(1, 2**length - 1), length)
        elif field == "RoomSize":
            chromosome += binary_encode(random.randint(1, 2**length - 1), length)
    return chromosome

def decode_chromosome(chromosome):
    decoded = {}
    start = 0
    for field, length in encoding_lengths.items():
        end = start + length
        value = int(chromosome[start:end], 2)
        decoded[field] = value
        start = end
    return decoded

# Checks constraint that professor is only assigned to one class at a time
def check_professor_constraints(timetable):
    violations = 0
    profSchedule = {}
    for cls in timetable:
        prof = cls["Professor"]
        day = cls["Day"]
        timeslot = cls["Timeslot"]
        if prof in profSchedule:
            if (day, timeslot) in profSchedule[prof]:
                violations += 1
            else:
                profSchedule[prof].append((day, timeslot))
        else:
            profSchedule[prof] = [(day, timeslot)]
    return violations

# Checks constraint that no section is double booked
def check_room_section_constraints(timetable):
    violations = 0
    sectionSchedule = {}
    for cls in timetable:
        section = cls["Section"]
        day = cls["Day"]
        timeslot = cls["Timeslot"]
        if section in sectionSchedule:
            if (day, timeslot) in sectionSchedule[section]:
                violations += 1
            else:
                sectionSchedule[section].append((day, timeslot))
        else:
            sectionSchedule[section] = [(day, timeslot)]
    return violations

# Checks constraint that professor can only be assigned to max 3 courses
def check_prof_course_constraints(timetable):
    profCoursesCount = {prof: 0 for prof in teacherList}
    for cls in timetable:
        prof = cls["Professor"]
        profCoursesCount[prof] += 1
    violations = sum(1 for count in profCoursesCount.values() if count > 3)
    return violations

# Checks constraint that each course has 2 lectures per week
# Checks constraint that each lecture is not on the same or adjacent day
def check_course_sched_constraints(timetable):
    courseLectureDays = {course["name"]: [] for course in courseData}
    for cls in timetable:
        courseName = cls["Course"]
        day = cls["Day"]
        if day not in courseLectureDays[courseName]:
            courseLectureDays[courseName].append(day)
    violations = 0
    for courseName, lectureDays in courseLectureDays.items():
        if len(lectureDays) != 2:
            violations += 1
        else:
            if abs(lectureDays[0] - lectureDays[1]) <= 1:
                violations += 1
    return violations

# Checks constraint that lab should be two consecutive slots
def check_lab_constraints(timetable):
    violations = 0
    labCourses = [course["name"] for course in courseData if course["type"] == "Lab"]
    for day in range(1, num_days + 1):
        for timeslot in range(1, timeslots_per_day - 1):
            for room in roomData:
                labOccupied = all(
                    cls["Course"] in labCourses
                    for cls in timetable
                    if cls["Day"] == day
                    and cls["Timeslot"] in [timeslot, timeslot + 1]
                    and cls["Classroom"] == room
                )
                otherCourseScheduled = any(
                    cls["Course"] not in labCourses
                    for cls in timetable
                    if cls["Day"] == day
                    and cls["Timeslot"] == timeslot + 1
                    and cls["Classroom"] == room
                )
                if labOccupied and not otherCourseScheduled:
                    continue
                elif labOccupied and otherCourseScheduled:
                    violations += 1
                elif not labOccupied and otherCourseScheduled:
                    violations += 1
                else:
                    continue
    return violations


#Check constraint that no section can have more than 5 courses
def check_section_course_limit(timetable):
    section_course_count = {section: 0 for section in sectionCourses.keys()}
    for cls in timetable:
        section = cls["Section"]
        if section_course_count[section] < 5:
            section_course_count[section] += 1
    violations = sum(1 for count in section_course_count.values() if count > 5)
    return violations

# Checks soft constraint that lab should be in afternoon and theory in morning
def check_sess_type_constraints(timetable):
    violations = 0
    for cls in timetable:
        courseInfo = [course for course in courseData if course["name"] == cls["Course"]]
        if not courseInfo:
            continue
        courseType = courseInfo[0]["type"]
        timeslot = cls["Timeslot"]
        if courseType == "Theory" and timeslot > 3:
            violations += 1
        if courseType == "Lab" and timeslot <= 3:
            violations += 1
    return violations

#Checks soft constraint that class should be held in the same classroom across the week
def check_room_consistency_constraints(timetable):
    violations = 0
    classToRoomMap = {}
    for cls in timetable:
        course = cls["Course"]
        room = cls["Classroom"]
        if course in classToRoomMap:
            if classToRoomMap[course] != room:
                violations += 1
        else:
            classToRoomMap[course] = room
    return violations

#Checks soft constraint that teacher should teach on the same floor
def check_teaching_blocks_constraints(timetable):
    violations = 0
    courseBlocks = {}
    for cls in timetable:
        course = cls["Course"]
        day = cls["Day"]
        timeslot = cls["Timeslot"]
        if course in courseBlocks:
            if timeslot != courseBlocks[course][-1] + 1:
                courseBlocks[course].append(timeslot)
        else:
            courseBlocks[course] = [timeslot]
    for course, blocks in courseBlocks.items():
        if len(blocks) > 1:
            violations += len(blocks) - 1
    return violations

# Check fitness value
def fitness_eval(timetable):
    majorPenalty = 1000
    minorPenalty = 10
    fitnessScore = 0
    totalViolations = 0
    #hard constraints
    totalViolations += check_classroom_constraints(timetable)
    totalViolations += check_professor_constraints(timetable)
    totalViolations += check_room_section_constraints(timetable)
    totalViolations += check_prof_course_constraints(timetable)
    totalViolations += check_course_sched_constraints(timetable)
    totalViolations += check_lab_constraints(timetable)

    fitnessScore = totalViolations * majorPenalty

    totalViolations = 0
    #soft constraints
    totalViolations += check_sess_type_constraints(timetable)
    totalViolations += check_room_consistency_constraints(timetable)
   # totalViolations += check_teaching_blocks_constraints(timetable)
    fitnessScore += totalViolations * minorPenalty

    fitnessScore *= -1

    return fitnessScore

def create_initial_timetable():
    timetable = []
    for day in range(1, num_days + 1):
        for timeslot in range(timeslots_per_day):
            for room, info in roomData.items():
                if info["isFree"][timeslot]:
                    courseSections = random.choice(list(sectionCourses.items()))
                    sectionName, section_courses = courseSections
                    courseName = random.choice(section_courses)
                    course = next(c for c in courseData if c["name"] == courseName)
                    courseType = course["type"]
                    professor = random.choice(teacherList)
                    if courseName in sectionCourses[sectionName]:
                        if courseType == "Lab":
                            if f"{courseName} - Lab" not in sectionCourses[sectionName]:
                                continue
                        info["isFree"][timeslot] = False
                        timetable.append(
                            {
                                "Day": day,
                                "Timeslot": timeslot + 1,
                                "Time": timeslot_to_time(
                                    timeslot + 1
                                ),  # Map timeslot to time
                                "Classroom": room,
                                "Course": courseName,
                                "Type": courseType,
                                "Section": sectionName,
                                "Professor": professor,
                            }
                        )
                        if course["weeklyLectures"] == 2:
                            if timeslot < timeslots_per_day - 1:
                                timeslot += 1
                            else:
                                day += 1
                                timeslot = 0
                            if day > num_days:
                                break
                            timetable.append(
                                {
                                    "Day": day,
                                    "Timeslot": timeslot + 1,
                                    "Time": timeslot_to_time(
                                        timeslot + 1
                                    ),  # Map timeslot to time
                                    "Classroom": room,
                                    "Course": courseName,
                                    "Type": courseType,
                                    "Section": sectionName,
                                    "Professor": professor,
                                }
                            )
    return timetable

#selection function
def selection(population, fitnessScores):
    totalFitness = sum(fitnessScores)
    probabilities = [fitness / totalFitness for fitness in fitnessScores]
    parent1 = roulette_wheel_selection(population, probabilities)
    parent2 = roulette_wheel_selection(population, probabilities)
    return parent1, parent2


def roulette_wheel_selection(population, probabilities):
    rand = random.random()
    cumulativeProbability = 0
    for i, probability in enumerate(probabilities):
        cumulativeProbability += probability
        if rand <= cumulativeProbability:
            return population[i]

#single point crossover
def crossover(parent1, parent2):
    minLength = min(len(parent1), len(parent2))
    if minLength < 2:
        return parent1, parent2
    crossoverPoint = random.randint(1, minLength - 1)
    offspring1 = parent1[:crossoverPoint] + parent2[crossoverPoint:]
    offspring2 = parent2[:crossoverPoint] + parent1[crossoverPoint:]
    return offspring1, offspring2

#mutate function for binary chromosomes
def mutation_binary(encoded_timetable):
    mutated = list(encoded_timetable)
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            mutated[i] = '1' if mutated[i] == '0' else '0'
    return ''.join(mutated)


def mutate_gene(gene):
    (
        course,
        courseType,
        section,
        sectionStrength,
        professor,
        firstLectureDay,
        firstLectureTimeslot,
        firstLectureRoom,
        firstLectureRoomSize,
        secondLectureDay,
        secondLectureTimeslot,
        secondLectureRoom,
        secondLectureRoomSize,
    ) = gene
    mutationIndex = random.randint(0, 11)
    if mutationIndex == 0:  # Course
        course = random.choice(courseData)["name"]
    elif mutationIndex == 1:  # Course Type (Theory/Lab)
        courseType = "Theory" if courseType == "Lab" else "Lab"
    elif mutationIndex == 2:  # Section
        section = random.choice(list(sectionInfo.keys()))
    elif mutationIndex == 4:  # Professor
        professor = random.choice(teacherList)
    elif mutationIndex in [5, 9]:  # Lecture Day (first or second)
        if mutationIndex == 5:
            firstLectureDay = mutate_lecture_day(firstLectureDay)
        else:
            secondLectureDay = mutate_lecture_day(secondLectureDay, firstLectureDay)
    elif mutationIndex in [6, 10]:  # Lecture Timeslot (first or second)
        if mutationIndex == 6:
            firstLectureTimeslot = random.randint(1, timeslots_per_day)
        else:
            secondLectureTimeslot = random.randint(1, timeslots_per_day)
    elif mutationIndex in [7, 11]:  # Lecture Room (first or second)
        if mutationIndex == 7:
            firstLectureRoom = mutate_lecture_room(courseType, sectionStrength)
            firstLectureRoomSize = roomData[firstLectureRoom]["capacity"]
        else:
            secondLectureRoom = mutate_lecture_room(courseType, sectionStrength)
            secondLectureRoomSize = roomData[secondLectureRoom]["capacity"]
    return (
        course,
        courseType,
        section,
        sectionStrength,
        professor,
        firstLectureDay,
        firstLectureTimeslot,
        firstLectureRoom,
        firstLectureRoomSize,
        secondLectureDay,
        secondLectureTimeslot,
        secondLectureRoom,
        secondLectureRoomSize,
    )

def mutate_lecture_day(currentDay, otherDay=None):
    if currentDay is None:
        return random.randint(1, num_days + 1)
    if otherDay is None:
        validDays = [day for day in range(1, num_days + 1) if abs(day - currentDay) > 1]
    else:
        validDays = [
            day
            for day in range(1, num_days + 1)
            if abs(day - currentDay) > 1 and abs(day - otherDay) > 1
        ]
    return random.choice(validDays) if validDays else currentDay

def mutate_lecture_room(courseType, sectionStrength):
    validRooms = [
        room
        for room, info in roomData.items()
        if info["capacity"] >= sectionStrength
        and (
            (courseType == "Theory" and info["capacity"] == 60)
            or (courseType == "Lab" and info["capacity"] == 120)
        )
    ]
    return random.choice(validRooms) if validRooms else random.choice(list(roomData.keys()))

def mutation(chromosome):
    mutatedChromosome = chromosome[:]
    for i in range(len(mutatedChromosome)):
        if random.random() < mutation_rate:
            gene = (
                mutatedChromosome[i]["Course"],
                mutatedChromosome[i]["Type"],
                mutatedChromosome[i]["Section"],
                sectionInfo[mutatedChromosome[i]["Section"]]["strength"],
                mutatedChromosome[i]["Professor"],
                mutatedChromosome[i]["Day"],
                mutatedChromosome[i]["Timeslot"],
                mutatedChromosome[i]["Classroom"],
                roomData[mutatedChromosome[i]["Classroom"]]["capacity"],
                None,
                None,
                None,
                None,
            )
            mutatedGene = mutate_gene(gene)
            (
                course,
                courseType,
                section,
                sectionStrength,
                professor,
                firstLectureDay,
                firstLectureTimeslot,
                firstLectureRoom,
                firstLectureRoomSize,
                secondLectureDay,
                secondLectureTimeslot,
                secondLectureRoom,
                secondLectureRoomSize,
            ) = mutatedGene
            mutatedChromosome[i] = {
                "Day": firstLectureDay,
                "Timeslot": firstLectureTimeslot,
                "Time": timeslot_to_time(
                    firstLectureTimeslot
                ),  # Map timeslot to time
                "Classroom": firstLectureRoom,
                "Course": course,
                "Type": courseType,
                "Section": section,
                "Professor": professor,
            }
    return mutatedChromosome

def genetic_algorithm(populationSize, maxGenerations):
    population = []
    for _ in range(populationSize):
        population.append(create_initial_timetable())
    binary_population = []
    for _ in range(populationSize):
        binary_population.append(create_binary_chromosome())
    generation = 0
    while generation < maxGenerations:
        fitnessScores = [fitness_eval(chromosome) for chromosome in population]
        if max(fitnessScores) == 0:
            return population[fitnessScores.index(max(fitnessScores))]
        matingPool = []
        for _ in range(populationSize // 2):
            parent1, parent2 = selection(population, fitnessScores)
            offspring1, offspring2 = crossover(parent1, parent2)
            matingPool.extend([offspring1, offspring2])
        population = [mutation(chromosome) for chromosome in matingPool]
        generation += 1
    return population[fitnessScores.index(min(fitnessScores))]

mutation_rate = 0.2
populationSize = 100
maxGenerations = 100

best_timetable = genetic_algorithm(populationSize, maxGenerations)
#best_fitness = fitness_eval(best_timetable)
print("Best Timetable:")
for cls in best_timetable:
    print(cls)
# print("Fitness Score:", best_fitness)

#binary_chromosome = create_binary_chromosome()
#decoded_chromosome = decode_chromosome(binary_chromosome)
#print("Binary Chromosome:", binary_chromosome)
#print("Decoded Chromosome:", decoded_chromosome)


Best Timetable:
{'Day': 4, 'Timeslot': 4, 'Time': '13:15', 'Classroom': 'C301', 'Course': 'COAL', 'Type': 'Theory', 'Section': 'G', 'Professor': 'Mr. Majid Hussain'}
{'Day': 5, 'Timeslot': 3, 'Time': '11:40', 'Classroom': 'C301', 'Course': 'COAL', 'Type': 'Theory', 'Section': 'C', 'Professor': 'Mr. Muhammad Ali'}
{'Day': 5, 'Timeslot': 1, 'Time': '08:30', 'Classroom': 'C502', 'Course': 'OOP', 'Type': 'Lab', 'Section': 'F', 'Professor': 'Mr. Owais Idrees'}
{'Day': 2, 'Timeslot': 6, 'Time': '16:25', 'Classroom': 'C305', 'Course': 'Algo', 'Type': 'Theory', 'Section': 'D', 'Professor': 'Ms. Labiba Fahad'}
{'Day': 2, 'Timeslot': 2, 'Time': '10:05', 'Classroom': 'C305', 'Course': 'DS - Lab', 'Type': 'Theory', 'Section': 'B', 'Professor': 'Mr. Almas'}
{'Day': 1, 'Timeslot': 1, 'Time': '08:30', 'Classroom': 'C501', 'Course': 'DB', 'Type': 'Lab', 'Section': 'F', 'Professor': 'Mr. Shahnawaz'}
{'Day': 5, 'Timeslot': 6, 'Time': '16:25', 'Classroom': 'C301', 'Course': 'DB', 'Type': 'Lab', 'Section'